In [2]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import r2_score

from reloading_utils import get_url

verbose = False

In [3]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv(get_url('data/reloading_data - CATRIDGES.csv'))
print(cartridges)

cartridges_to_process = cartridges.loc[:, ['title', 'volume', 'diameter']].to_numpy()
print(cartridges_to_process)

             title  volume  diameter
0   9mm parabellum    0.74     0.356
1       38 special    1.30     0.358
2       357 magnum    1.36     0.358
3  7.62x54 russian    3.47     0.310
4    38 special +P    1.18     0.358
5  7.62x39 russian    2.03     0.311
[['9mm parabellum' 0.74 0.356]
 ['38 special' 1.3 0.358]
 ['357 magnum' 1.36 0.358]
 ['7.62x54 russian' 3.47 0.31]
 ['38 special +P' 1.18 0.358]
 ['7.62x39 russian' 2.03 0.311]]


In [4]:
def process_data_file(c, v, a):
    filename = f'data/reloading_data - %s.csv' % c
    try:
        d = pd.read_csv(get_url(filename))
        d['cartridge_caliber'] = c
        d['cartridge_volume'] = v
        d['cartridge_diameter'] = a
        return d
    except:
        print("ERROR: Cannot process:", filename)
        return None

frames = [ process_data_file(c, v, a) for c, v, a in cartridges_to_process ]
data = pd.concat(frames, ignore_index=True)

# print(data.head())  # DEBUG
print('data.shape', data.shape)

data.shape (409, 16)


In [5]:
data.tail()

,powder,start,volume cc,auto disk,lee dipper,velocity,never exceed,velocity max,press,units,min oal,mm,press_psi,cartridge_caliber,cartridge_volume,cartridge_diameter
404,H335,30.0,NaN,NaN,NaN,2219.0,31.5,2408.0,40900.0,CUP,2.15,54.61,NaN,7.62x39 russian,2.03,0.311
405,H4198,24.5,NaN,NaN,NaN,2190.0,26.5,2378.0,40400.0,CUP,2.15,54.61,NaN,7.62x39 russian,2.03,0.311
406,H4895,28.0,NaN,NaN,NaN,2171.0,29.0,2249.0,33600.0,CUP,2.15,54.61,NaN,7.62x39 russian,2.03,0.311
407,IMR4198,22.6,NaN,NaN,NaN,2035.0,24.0,2250.0,42500.0,CUP,2.15,54.61,NaN,7.62x39 russian,2.03,0.311
408,125 grain SIE SPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.62x39 russian,2.03,0.311


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   powder              409 non-null    object 
 1   start               374 non-null    float64
 2   volume cc           15 non-null     float64
 3   auto disk           2 non-null      float64
 4   lee dipper          2 non-null      float64
 5   velocity            374 non-null    float64
 6   never exceed        374 non-null    float64
 7   velocity max        374 non-null    float64
 8   press               322 non-null    float64
 9   units               322 non-null    object 
 10  min oal             374 non-null    float64
 11  mm                  374 non-null    float64
 12  press_psi           2 non-null      float64
 13  cartridge_caliber   409 non-null    object 
 14  cartridge_volume    409 non-null    float64
 15  cartridge_diameter  409 non-null    float64
dtypes: float

In [7]:
# reloading_data - powder burning rate chart
powder_rates = pd.read_csv(get_url('data/reloading_data - powder burning rate chart.csv'))
print(powder_rates.head())

#
# verify the data
#
unique_powder_names = powder_rates.powder.unique()
print(len(unique_powder_names), len(powder_rates))
assert np.all(unique_powder_names == powder_rates.powder), "powder_rates is not unique"

        powder  rate  type  f0 note
0   A NITRO100     1   NaN NaN  NaN
1       v-N310     1   NaN NaN  NaN
2     VEC BA10     2   NaN NaN  NaN
3     ALLNT E3     2   NaN NaN  NaN
4  WIN AA LITE     2   NaN NaN  NaN
184 184


In [8]:
# add a "bullet type" column

orig_columns = [
    'powder', 'start', 'velocity', 'never exceed', 'velocity max', 'press', 'units', 'min oal',
    'cartridge_caliber', 'cartridge_volume', 'cartridge_diameter'
    ]
new_columns  = [
    'bul_weight', 'bul_type', 
    'powder', 's_load', 's_velocity', 'ne_load', 'ne_velocity', 'press', 'units', 'oal',
    'cartridge_caliber', 'cartridge_volume', 'cartridge_diameter'
    ]

lee = pd.DataFrame(columns = new_columns)

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = tuple(data.loc[pos, orig_columns].array)
    # print(123, '>>>', line) # DEBUG
    if type(line[0]) is float:
        continue
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (float(bul_weight), bul_type, *line)
        lee.loc[len(lee.index)] = line
print('lee.shape', lee.shape)

lee.shape (374, 13)


In [9]:
# check if some powder names are not known from the burn rate table
unknown_powders = np.setdiff1d(lee.powder.unique(), unique_powder_names)
for p in unknown_powders:
    print("'%s' is not found in powder burning rate chart" % p)

In [10]:
def powder_id(x):
    try:
        return np.where(powder_rates.powder == x)[0][0]
    except :
        print (x)

def bul_type_id(x):
    return np.where(lee.bul_type.unique() == x)[0][0]


In [11]:
lee['powder_id'] = [ powder_id(x) for x in lee.powder ]
lee['bul_type_id'] = [ bul_type_id(x) for x in lee.bul_type ]
lee['powder_rate'] = list(powder_rates.rate[lee.powder_id])

lee.to_csv('lee.csv')
lee.info()
lee.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 373
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bul_weight          374 non-null    float64
 1   bul_type            374 non-null    object 
 2   powder              374 non-null    object 
 3   s_load              374 non-null    float64
 4   s_velocity          374 non-null    float64
 5   ne_load             374 non-null    float64
 6   ne_velocity         374 non-null    float64
 7   press               322 non-null    float64
 8   units               322 non-null    object 
 9   oal                 374 non-null    float64
 10  cartridge_caliber   374 non-null    object 
 11  cartridge_volume    374 non-null    float64
 12  cartridge_diameter  374 non-null    float64
 13  powder_id           374 non-null    int64  
 14  bul_type_id         374 non-null    int64  
 15  powder_rate         374 non-null    int64  
dtypes: float

,bul_weight,bul_type,powder,s_load,s_velocity,ne_load,ne_velocity,press,units,oal,cartridge_caliber,cartridge_volume,cartridge_diameter,powder_id,bul_type_id,powder_rate
0,115.0,jacketed,v-N350,5.4,1129.0,6.5,1293.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,58,0,20
1,115.0,jacketed,v-3N37,5.6,1129.0,6.5,1289.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,54,0,19
2,115.0,jacketed,v-N330,4.5,1076.0,5.4,1227.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,40,0,15
3,115.0,jacketed,v-N340,4.8,1129.0,5.4,1220.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,46,0,18
4,115.0,jacketed,wSUPER-FLD,4.9,1060.0,5.7,1195.0,31900.0,PSI,1.169,9mm parabellum,0.74,0.356,55,0,19


In [12]:
# print(lee)
# lee.to_csv('lee.csv')
# print("saved...")

if False:
    for p in ['HP38', 'H110']:
        arr = lee[lee.powder == p][['powder', 'powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter']].to_numpy()
        print(arr[0])

    print(lee[lee.powder == 'H110'][['powder', 'powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter']])
    print('-------------')
    print(lee[lee.powder == 'H110'])

In [13]:
print(powder_rates[powder_rates.powder == 'HP38'].to_numpy())
print(lee.bul_weight.unique())
print(powder_rates[powder_rates.powder == 'ACCUR 1680'])

[['HP38' 7 nan nan nan]]
[115. 125. 147.  77. 130. 135. 110.  90. 146. 123. 150.]
        powder  rate  type  f0 note
82  ACCUR 1680    29   NaN NaN  NaN


In [15]:
lee_clean = lee.dropna()
lee_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 4 to 373
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bul_weight          322 non-null    float64
 1   bul_type            322 non-null    object 
 2   powder              322 non-null    object 
 3   s_load              322 non-null    float64
 4   s_velocity          322 non-null    float64
 5   ne_load             322 non-null    float64
 6   ne_velocity         322 non-null    float64
 7   press               322 non-null    float64
 8   units               322 non-null    object 
 9   oal                 322 non-null    float64
 10  cartridge_caliber   322 non-null    object 
 11  cartridge_volume    322 non-null    float64
 12  cartridge_diameter  322 non-null    float64
 13  powder_id           322 non-null    int64  
 14  bul_type_id         322 non-null    int64  
 15  powder_rate         322 non-null    int64  
dtypes: float

In [16]:
lee_dummies = pd.get_dummies(lee_clean)
lee_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 4 to 373
Data columns (total 76 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   bul_weight                         322 non-null    float64
 1   s_load                             322 non-null    float64
 2   s_velocity                         322 non-null    float64
 3   ne_load                            322 non-null    float64
 4   ne_velocity                        322 non-null    float64
 5   press                              322 non-null    float64
 6   oal                                322 non-null    float64
 7   cartridge_volume                   322 non-null    float64
 8   cartridge_diameter                 322 non-null    float64
 9   powder_id                          322 non-null    int64  
 10  bul_type_id                        322 non-null    int64  
 11  powder_rate                        322 non-null    int64  

In [52]:
keep_looping = True
total_loops = 0
max_loops = 20
DEGREE = 2

y_keys = ['s_load', 'ne_load']
x_keys = [
    'bul_weight', 'powder_rate', 'cartridge_volume', 'cartridge_diameter', 'bul_type_id',
    's_velocity', 'ne_velocity',
#    'cartridge_caliber_357 magnum',  
#    'cartridge_caliber_38 special',  
#    'cartridge_caliber_38 special +P',  
#    'cartridge_caliber_7.62x39 russian',  
#    'cartridge_caliber_9mm parabellum',
#    'bul_type_FRANGIBLE',
#    'bul_type_HDY SST',
#    'bul_type_SPR SP',
#    'bul_type_XTP',
#    'bul_type_barnes x solid',
#    'bul_type_copper plated',
#    'bul_type_jacketed',
#    'bul_type_lead', 
]


def compute_load(powder, weight, caliber):
    pwd, rate, s_velocity, ne_velocity = lee[lee.powder == powder][['powder_id', 'powder_rate', 's_velocity', 'ne_velocity']].to_numpy()[0]
    print('powder id', pwd, 'rate', rate)
    vol, diam = cartridges[cartridges.title == caliber][['volume', 'diameter']].to_numpy()[0]
    print('volume', vol, 'diameter', diam)
    
    if False:
        if caliber == '357 magnum':
            cartridge_caliber = [1, 0, 0, 0, 0]
        elif caliber == '38 special':
            cartridge_caliber = [0, 1, 0, 0, 0]
        elif caliber == 'caliber_38 special +P':
            cartridge_caliber = [0, 0, 1, 0, 0]
        elif caliber == '7.62x39 russian':
            cartridge_caliber = [0, 0, 0, 1, 0]
        elif caliber == '9mm parabellum':
            cartridge_caliber = [0, 0, 0, 0, 1]
        else:
            raise "Unknown caliber"
    
    actual_x = np.array(
        list([weight, rate, vol, diam, x, s_velocity, ne_velocity ]  for x in range(len(lee.bul_type.unique())))
    )
    
    #hot_one = np.eye(len(lee.bul_type.unique()))
    #actual_x = np.concatenate((actual_x, hot_one), axis=1)
    
    print('X\n', actual_x)
    actual_x_poly = poly.fit_transform(actual_x)
    actual_y_ = clf.predict(actual_x_poly)
    print(lee.bul_type.unique())
    print('Y\n', actual_y_)

while keep_looping and total_loops < max_loops:
    msk = np.random.rand(len(lee_clean)) < 0.8
    train = lee_clean[msk]
    test = lee_clean[~msk]

    train_x = np.asanyarray(train[x_keys])
    train_y = np.asanyarray(train[y_keys])

    test_x = np.asanyarray(test[x_keys])
    test_y = np.asanyarray(test[y_keys])
    print('train_x', train_x.shape)

    poly = PolynomialFeatures(degree=DEGREE)
    train_x_poly = poly.fit_transform(train_x)
    print('train_x_poly', train_x_poly.shape)

    clf = linear_model.LinearRegression()
    train_y_ = clf.fit(train_x_poly, train_y)
    # The coefficients
    if False:
        print ('Coefficients: ', clf.coef_)
        print ('Intercept: ',clf.intercept_)

    test_x_poly = poly.fit_transform(test_x)
    test_y_ = clf.predict(test_x_poly)

    total_loops += 1

    if verbose:
        print('Loop: %d ------------' % total_loops)
        print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
        print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
        print("R2-score: %.2f" % r2_score(test_y_ , test_y) )
    
    err = np.mean(np.absolute(test_y_ - test_y))
    keep_looping = err > 0.95
                  
# end of while
print('Loops done: %d' % total_loops)
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , test_y) )

assert(keep_looping == False)

train_x (263, 7)
train_x_poly (263, 36)
train_x (256, 7)
train_x_poly (256, 36)
Loops done: 2
Mean absolute error: 0.71
Residual sum of squares (MSE): 1.00
R2-score: 0.99


In [43]:
print(lee.bul_type.unique())
print(lee.bul_weight.unique())

print(cartridges.loc[:, ['title', 'volume', 'diameter']])

['jacketed' 'lead' 'copper plated' 'XTP' 'barnes x solid' 'FRANGIBLE'
 'HDY SST' 'SPR SP']
[115. 125. 147.  77. 130. 135. 110.  90. 146. 123. 150.]
             title  volume  diameter
0   9mm parabellum    0.74     0.356
1       38 special    1.30     0.358
2       357 magnum    1.36     0.358
3  7.62x54 russian    3.47     0.310
4    38 special +P    1.18     0.358
5  7.62x39 russian    2.03     0.311


In [53]:
compute_load('ACCUR #2', 147, '38 special')

powder id 23.0 rate 9.0
volume 1.3 diameter 0.358
X
 [[1.470e+02 9.000e+00 1.300e+00 3.580e-01 0.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 1.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 2.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 3.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 4.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 5.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 6.000e+00 9.710e+02 1.088e+03]
 [1.470e+02 9.000e+00 1.300e+00 3.580e-01 7.000e+00 9.710e+02 1.088e+03]]
['jacketed' 'lead' 'copper plated' 'XTP' 'barnes x solid' 'FRANGIBLE'
 'HDY SST' 'SPR SP']
Y
 [[5.34116112 6.53563288]
 [5.31117792 6.52760748]
 [5.50905473 6.68181391]
 [5.93479157 6.99825218]
 [6.58838844 7.47692229]
 [7.46984533 8.11782423]
 [8.57916225 8.92095802]
 [9.91633918 9.88632366]]


In [46]:
compute_load('HS6', 125, '9mm parabellum')

powder id 43 rate 18
volume 0.74 diameter 0.356
X
 [[125.     18.      0.74    0.356   0.   ]
 [125.     18.      0.74    0.356   1.   ]
 [125.     18.      0.74    0.356   2.   ]
 [125.     18.      0.74    0.356   3.   ]
 [125.     18.      0.74    0.356   4.   ]
 [125.     18.      0.74    0.356   5.   ]
 [125.     18.      0.74    0.356   6.   ]
 [125.     18.      0.74    0.356   7.   ]]
['jacketed' 'lead' 'copper plated' 'XTP' 'barnes x solid' 'FRANGIBLE'
 'HDY SST' 'SPR SP']
Y
 [[4.93440953 5.52835299]
 [4.81730537 5.49715872]
 [4.96948495 5.65980994]
 [5.39094827 6.01630665]
 [6.08169532 6.56664884]
 [7.0417261  7.31083651]
 [8.27104062 8.24886967]
 [9.76963888 9.38074831]]


In [54]:
compute_load('HS6', 147, '38 special')

powder id 43.0 rate 18.0
volume 1.3 diameter 0.358
X
 [[1.470e+02 1.800e+01 1.300e+00 3.580e-01 0.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 1.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 2.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 3.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 4.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 5.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 6.000e+00 1.117e+03 1.170e+03]
 [1.470e+02 1.800e+01 1.300e+00 3.580e-01 7.000e+00 1.117e+03 1.170e+03]]
['jacketed' 'lead' 'copper plated' 'XTP' 'barnes x solid' 'FRANGIBLE'
 'HDY SST' 'SPR SP']
Y
 [[ 7.28250055  8.08428122]
 [ 7.28139097  8.16163719]
 [ 7.50814141  8.401225  ]
 [ 7.96275188  8.80304464]
 [ 8.64522237  9.36709611]
 [ 9.55555288 10.09337944]
 [10.69374342 10.9818946 ]
 [12.05979398 12.0326416 ]]


In [55]:
compute_load('H110', 123, '7.62x39 russian')

powder id 75.0 rate 26.0
volume 2.03 diameter 0.311
X
 [[1.230e+02 2.600e+01 2.030e+00 3.110e-01 0.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 1.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 2.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 3.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 4.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 5.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 6.000e+00 1.881e+03 1.966e+03]
 [1.230e+02 2.600e+01 2.030e+00 3.110e-01 7.000e+00 1.881e+03 1.966e+03]]
['jacketed' 'lead' 'copper plated' 'XTP' 'barnes x solid' 'FRANGIBLE'
 'HDY SST' 'SPR SP']
Y
 [[18.66965275 20.1166724 ]
 [18.04497354 19.68168497]
 [17.64815435 19.40892937]
 [17.47919518 19.29840561]
 [17.53809604 19.35011369]
 [17.82485693 19.56405361]
 [18.33947783 19.94022538]
 [19.08195876 20.47862897]]


In [98]:
load_rate = lee[['s_load', 'ne_load']].copy()
load_rate['rate'] = (load_rate.ne_load / load_rate.s_load)

In [99]:
load_rate.describe()

,s_load,ne_load,rate
count,374.000000,374.000000,374.000000
mean,8.188503,9.171658,1.140950
std,8.124903,8.829368,0.082863
min,2.500000,2.700000,1.035714
25%,3.925000,4.600000,1.106383
50%,4.900000,5.650000,1.120000
75%,6.400000,7.200000,1.155460
max,49.200000,54.000000,1.800000


In [100]:
lee.groupby('powder').bul_type.count()

powder
A NITRO100     7
ACCUR #2      13
ACCUR #5      13
ACCUR #7       7
ACCUR 1680     4
              ..
v-N320         7
v-N330         3
v-N340         7
v-N350         7
wSUPER-FLD     3
Name: bul_type, Length: 66, dtype: int64

In [ ]:
lee.groupby('powder').count()

In [ ]:
compute_load('ACCUR #2', 123, '7.62x39 russian')

In [ ]:
compute_load('ACCUR 1680', 123, '7.62x39 russian')

In [81]:
e1 = np.eye(3)
e2 = np.eye(3)
e1, e2

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [82]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
np.concatenate((a, b), axis=0)

array([[1, 2],
       [3, 4],
       [5, 6]])

In [83]:
a

array([[1, 2],
       [3, 4]])

In [84]:
b

array([[5, 6]])

In [85]:
np.concatenate((e1, e2), axis=1)

array([[1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.]])